In [3]:
from texas_gerrymandering_hb4.config import PLANC2308_SHP_FILE

import geopandas as gpd

gdf = gpd.read_file(PLANC2308_SHP_FILE)

2025-08-24 19:03:28.980 | INFO     | texas_gerrymandering_hb4.config:<module>:11 - PROJ_ROOT path is: /home/aimlexpert/Documents/GitHub/texas-gerrymandering-HB4
